## Appointments for Cortesy

In [3]:
import pandas as pd
import numpy as np
import streamlit as st
from dictionaries import obter_dicionarios
from vmb import criando_df_final_Rentabilidade
pd.options.mode.chained_assignment = None
#from mongo import *

## Treating Appointments:

# Function to Treat Appointments to cortesy analysis
# Concat Bases

appointments_concat = "C:/Users/novo1/OneDrive/Desktop/Dev/Rentabilidade Anual/Bases/Agendamento/2024/appointments_2024_concat.csv"
vmb_concat = "C:/Users/novo1/OneDrive/Desktop/Dev/Rentabilidade Anual/Bases/Venda Mesal Bruta/2024/vmb_2024_concat.csv"
ano = 2024

appointments = pd.read_csv(appointments_concat,low_memory=False)
vmb_concat = pd.read_csv(vmb_concat,low_memory=False)

# Select relevant columns
appointments_columns = ['ID agendamento', 'ID cliente',
                       'Unidade do agendamento',
                       'Procedimento', 'Data', 'Status']

appointments = appointments[appointments_columns].copy()

# Add quantity column
appointments["Quantidade"] = 1

# Filter out unwanted branches
branches_to_desconsider = ['PLÁSTICA', 'HOMA', 'PRAIA GRANDE','RIBEIRÃO PRETO', 'BELO HORIZONTE']
appointments = appointments[~appointments['Unidade do agendamento'].isin(branches_to_desconsider)]

# Get dictionaries
Appointments_dic, Sales_dic, Month_dic, duration_dic, all_costs_2024, all_costs_2025 = obter_dicionarios()

# Standardize procedures
appointments["Procedimento_padronizado"] = appointments['Procedimento'].map(Appointments_dic)

# Dropping unmapped values and other procedures that we dont look: 
appointments = appointments.loc[appointments['Procedimento_padronizado'] != "UNMAPPED"]
appointments = appointments.loc[~appointments['Procedimento_padronizado'].isin(['TATUAGEM', 'DEPILAÇÃO', 'PRÉ TRATAMENTO'])]

# Validation of unmapped procedures
nan_procedures = appointments.loc[
    appointments['Procedimento_padronizado'].isna(),
    'Procedimento'
].unique().tolist()

if nan_procedures:
    print("Procedimentos sem Padronização!")
    print("Pedir ao Thales para corrigir o dicionário para os seguintes Procedimentos:")
    for proc in nan_procedures:
        print(f"- {proc}")
    print(f"\nTotal de procedimentos não mapeados: {len(nan_procedures)}")
else:
    print("Todos os Procedimentos foram Mapeados com sucesso!")

appointments["Cortesia?"] = appointments['Procedimento'].str.contains("CORTESIA", case=False, na=False)

appointments_columns = ['ID agendamento', 'ID cliente','Unidade do agendamento','Procedimento_padronizado', "Quantidade", 'Data', 'Status','Cortesia?']

appointments = appointments[appointments_columns]

appointments["Tempo"] = appointments['Procedimento_padronizado'].map(duration_dic)

# Verifying if there is any procedure that we don't inform the time:

no_time_procedures = appointments.loc[appointments['Tempo'].isna(),'Procedimento_padronizado'].unique().tolist()

if no_time_procedures:
    print("Há Procedimentos que não tem o tempo Mapeado!")
    print("Pedir ao Thales para corrigir o dicionário de tempo para os seguintes Procedimentos:")
    for var in no_time_procedures:
        print(f"- {var}")
        print(f"\nTotal de procedimentos não mapeados: {len(no_time_procedures)}")
else: 
    print("Todos os tempos foram mapeados com sucesso!")

# Convert string to timedelta, then extract total minutes
appointments['Tempo'] = (
    pd.to_timedelta(appointments['Tempo'])  # Convert to timedelta
    .dt.total_seconds()                     # Convert to total seconds
    .div(60)                                # Convert seconds to minutes
    .astype(int)                            # Convert to integer
)

# this df will be used for analyse the real idle rate
df_appointments_general = appointments.copy()

#df to analyze only the courtesy served
appointments_cortesy = appointments.loc[appointments['Cortesia?'] == True]
appointments_cortesy = appointments_cortesy.loc[appointments_cortesy['Status'] == "Atendido"]

appointments_cortesy_columns = ['ID agendamento', 'ID cliente','Unidade do agendamento','Procedimento_padronizado', "Quantidade", 'Data', 'Status','Tempo', 'Cortesia?']

appointments_cortesy = appointments_cortesy[appointments_cortesy_columns]

# Bringing all costs of the procedures: 
    # Colunas de custo: 
def get_cost(row, cost_type):
    if ano == 2024:
        return all_costs_2024.get(row['Procedimento_padronizado'], {}).get(cost_type, 0)
    elif ano == 2025:
        return all_costs_2025.get(row['Procedimento_padronizado'], {}).get(cost_type, 0)
    else:
        return 0
    
appointments_cortesy['Custo Direto'] = appointments_cortesy.apply(lambda row: get_cost(row, 'CUSTO TOTAL'), axis=1)

# Verifying if there is any procedure that we don't inform the Direct cost:
no_cost_procedures = appointments_cortesy.loc[appointments_cortesy['Custo Direto'].isna(),'Procedimento_padronizado'].unique().tolist()

if no_cost_procedures: 
    print("Procedimentos sem custo!")
    print("Pedir ao Thales para corrigir o dicionário de custos para os seguintes Procedimentos:")
    for prod in no_cost_procedures:
        print(f"- {prod}")
        print(f"\nTotal de procedimentos não mapeados: {len(no_cost_procedures)}")
else: 
    print("Todos os custos foram mapeados com sucesso!")




appointments_cortesy['Data'] = pd.to_datetime(appointments_cortesy['Data'], format='%d/%m/%Y')
appointments_cortesy['Mês'] = appointments_cortesy['Data'].dt.month  
appointments_cortesy['Data'] = appointments_cortesy['Data'].dt.strftime('%d/%m/%Y')

appointments_cortesy['Mês'] = appointments_cortesy['Mês'].map(Month_dic)

appointments_cortesy_columns = ['ID agendamento', 'ID cliente','Unidade do agendamento',
                                'Procedimento_padronizado', "Quantidade",'Data','Mês', 
                                'Status','Custo Direto','Tempo']

appointments_cortesy = appointments_cortesy[appointments_cortesy_columns]

# Calling the others dataframes that we need (VMB and idle rate)
custo_fixo = pd.read_excel('C:/Users/novo1/OneDrive/Desktop/Dev/Rentabilidade Anual/Bases/teste_para_cortesia/CF-txSala.xlsx')
vmb_concat = pd.read_csv("C:/Users/novo1/OneDrive/Desktop/Dev/Rentabilidade Anual/Bases/teste_para_cortesia/vmb_2024_concat.csv", low_memory=False)
df_taxas = pd.read_excel('C:/Users/novo1/OneDrive/Desktop/Dev/Rentabilidade Anual/Bases/teste_para_cortesia/CF-txSala.xlsx',sheet_name="IMP + CART")        

df_final = criando_df_final_Rentabilidade(custo_fixo,vmb_concat,df_taxas)

df_final

Todos os Procedimentos foram Mapeados com sucesso!
Todos os tempos foram mapeados com sucesso!
Todos os custos foram mapeados com sucesso!


,ID orçamento,ID cliente,Status,Mês venda,Ano de venda,Unidade,Valor líquido,Procedimento_padronizado,Quantidade,Valor tabela item,...,Custo Insumos,Custo Mod,Custo Direto Total,Custo Sobre Venda Final,Cortesia?,Taxa Sala (Min),Taxa Ociosidade (Min),Custo Fixo,Custo Total,Lucro
0,342416,894220.0,Finalizado,Janeiro,2024.0,CAMPINAS,0.0,PRÓ-LIPO,1,290.0,...,3.41,20.0,46.51,0.000000,True,0.601016,3.208021,57.135565,103.645565,-103.645565
1,344454,404745.0,Finalizado,Janeiro,2024.0,CAMPINAS,1380.0,LAVIEEN,4,345.0,...,167.48,0.0,392.48,363.816150,False,0.601016,3.208021,228.542260,984.838411,395.161589
2,344941,656876.0,Finalizado,Janeiro,2024.0,JARDINS,0.0,SCULPTRA,1,2500.0,...,21.49,75.0,1034.49,659.087229,False,2.136979,3.284508,162.644617,1856.221846,643.778154
3,345688,897208.0,Finalizado,Janeiro,2024.0,MOEMA,100.0,LIMPEZA DE PELE,2,50.0,...,4.12,46.6,69.40,26.363489,False,1.144422,3.750417,587.380634,683.144123,-583.144123
4,346845,847721.0,Finalizado,Janeiro,2024.0,CAMPINAS,90.0,LIMPEZA DE PELE,1,90.0,...,2.06,23.3,34.70,23.727140,False,0.601016,3.208021,228.542260,286.969400,-196.969400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68862,423772,408674.0,Finalizado,Dezembro,2024.0,JARDINS,0.0,SECAGEM,1,190.0,...,2.11,15.0,21.59,27.420627,False,2.136979,3.070631,156.228309,205.238936,-85.238936
68863,423773,847094.0,Finalizado,Dezembro,2024.0,SANTOS,2750.0,ULTRAFORMER FULLFACE,1,2300.0,...,13.96,40.0,736.46,342.757838,False,0.927813,2.482690,204.630174,1283.848012,216.151988
68864,423773,847094.0,Finalizado,Dezembro,2024.0,SANTOS,0.0,ULTRAFORMER PESCOÇO,1,1500.0,...,13.96,30.0,463.96,182.804180,False,0.927813,2.482690,136.420116,783.184297,16.815703
68865,423773,847094.0,Finalizado,Dezembro,2024.0,SANTOS,0.0,ULTRAFORMER PAPADA,1,1049.0,...,13.96,30.0,253.96,102.827351,False,0.927813,2.482690,102.315087,459.102439,-9.102439


In [2]:
def treating_appointments_cortesy(appointments_concat):

    appointments_concat = "C:/Users/novo1/OneDrive/Desktop/Dev/Rentabilidade Anual/Bases/Agendamento/2024/appointments_2024_concat.csv"
    vmb_concat = "C:/Users/novo1/OneDrive/Desktop/Dev/Rentabilidade Anual/Bases/Venda Mesal Bruta/2024/vmb_2024_concat.csv"
    ano = 2024

    appointments = pd.read_csv(appointments_concat,low_memory=False)
    vmb_concat = pd.read_csv(vmb_concat,low_memory=False)

    # Select relevant columns
    appointments_columns = ['ID agendamento', 'ID cliente',
                        'Unidade do agendamento',
                        'Procedimento', 'Data', 'Status']

    appointments = appointments[appointments_columns].copy()

    # Add quantity column
    appointments["Quantidade"] = 1

    # Filter out unwanted branches
    branches_to_desconsider = ['PLÁSTICA', 'HOMA', 'PRAIA GRANDE','RIBEIRÃO PRETO', 'BELO HORIZONTE']
    appointments = appointments[~appointments['Unidade do agendamento'].isin(branches_to_desconsider)]

    # Get dictionaries
    Appointments_dic, Sales_dic, Month_dic, duration_dic, all_costs_2024, all_costs_2025 = obter_dicionarios()

    # Standardize procedures
    appointments["Procedimento_padronizado"] = appointments['Procedimento'].map(Appointments_dic)

    # Dropping unmapped values and other procedures that we dont look: 
    appointments = appointments.loc[appointments['Procedimento_padronizado'] != "UNMAPPED"]
    appointments = appointments.loc[~appointments['Procedimento_padronizado'].isin(['TATUAGEM', 'DEPILAÇÃO', 'PRÉ TRATAMENTO'])]

    # Validation of unmapped procedures
    nan_procedures = appointments.loc[
        appointments['Procedimento_padronizado'].isna(),
        'Procedimento'
    ].unique().tolist()

    if nan_procedures:
        print("Procedimentos sem Padronização!")
        print("Pedir ao Thales para corrigir o dicionário para os seguintes Procedimentos:")
        for proc in nan_procedures:
            print(f"- {proc}")
        print(f"\nTotal de procedimentos não mapeados: {len(nan_procedures)}")
    else:
        print("Todos os Procedimentos foram Mapeados com sucesso!")

    appointments["Cortesia?"] = appointments['Procedimento'].str.contains("CORTESIA", case=False, na=False)

    appointments_columns = ['ID agendamento', 'ID cliente','Unidade do agendamento','Procedimento_padronizado', "Quantidade", 'Data', 'Status','Cortesia?']

    appointments = appointments[appointments_columns]

    appointments["Tempo"] = appointments['Procedimento_padronizado'].map(duration_dic)

    # Verifying if there is any procedure that we don't inform the time:

    no_time_procedures = appointments.loc[appointments['Tempo'].isna(),'Procedimento_padronizado'].unique().tolist()

    if no_time_procedures:
        print("Há Procedimentos que não tem o tempo Mapeado!")
        print("Pedir ao Thales para corrigir o dicionário de tempo para os seguintes Procedimentos:")
        for var in no_time_procedures:
            print(f"- {var}")
            print(f"\nTotal de procedimentos não mapeados: {len(no_time_procedures)}")
    else: 
        print("Todos os tempos foram mapeados com sucesso!")

    # Convert string to timedelta, then extract total minutes
    appointments['Tempo'] = (
        pd.to_timedelta(appointments['Tempo'])  # Convert to timedelta
        .dt.total_seconds()                     # Convert to total seconds
        .div(60)                                # Convert seconds to minutes
        .astype(int)                            # Convert to integer
    )

    # this df will be used for analyse the real idle rate
    df_appointments_general = appointments.copy()

    #df to analyze only the courtesy served
    appointments_cortesy = appointments.loc[appointments['Cortesia?'] == True]
    appointments_cortesy = appointments_cortesy.loc[appointments_cortesy['Status'] == "Atendido"]

    appointments_cortesy_columns = ['ID agendamento', 'ID cliente','Unidade do agendamento','Procedimento_padronizado', "Quantidade", 'Data', 'Status','Tempo', 'Cortesia?']

    appointments_cortesy = appointments_cortesy[appointments_cortesy_columns]

    # Bringing all costs of the procedures: 
        # Colunas de custo: 
    def get_cost(row, cost_type):
        if ano == 2024:
            return all_costs_2024.get(row['Procedimento_padronizado'], {}).get(cost_type, 0)
        elif ano == 2025:
            return all_costs_2025.get(row['Procedimento_padronizado'], {}).get(cost_type, 0)
        else:
            return 0
        
    appointments_cortesy['Custo Direto'] = appointments_cortesy.apply(lambda row: get_cost(row, 'CUSTO TOTAL'), axis=1)

    # Verifying if there is any procedure that we don't inform the Direct cost:
    no_cost_procedures = appointments_cortesy.loc[appointments_cortesy['Custo Direto'].isna(),'Procedimento_padronizado'].unique().tolist()

    if no_cost_procedures: 
        print("Procedimentos sem custo!")
        print("Pedir ao Thales para corrigir o dicionário de custos para os seguintes Procedimentos:")
        for prod in no_cost_procedures:
            print(f"- {prod}")
            print(f"\nTotal de procedimentos não mapeados: {len(no_cost_procedures)}")
    else: 
        print("Todos os custos foram mapeados com sucesso!")




    appointments_cortesy['Data'] = pd.to_datetime(appointments_cortesy['Data'], format='%d/%m/%Y')
    appointments_cortesy['Mês'] = appointments_cortesy['Data'].dt.month  
    appointments_cortesy['Data'] = appointments_cortesy['Data'].dt.strftime('%d/%m/%Y')

    appointments_cortesy['Mês'] = appointments_cortesy['Mês'].map(Month_dic)

    appointments_cortesy_columns = ['ID agendamento', 'ID cliente','Unidade do agendamento',
                                    'Procedimento_padronizado', "Quantidade",'Data','Mês', 
                                    'Status','Custo Direto','Tempo']

    appointments_cortesy = appointments_cortesy[appointments_cortesy_columns]

    return appointments_cortesy

## Sales for Cortesy